# Import Required Packages

In [1]:
%%capture
!pip install -q -U keras-tuner

In [2]:
# basic packages
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn packages
from sklearn.metrics import (confusion_matrix, 
                             classification_report)

# tensorflow packages
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dense, 
                                     LSTM, 
                                     BatchNormalization,
                                     Input)
from tensorflow.keras.optimizers import (Adam, 
                                         AdamW)
from tensorflow.keras.losses import (SparseCategoricalCrossentropy,
                                     CategoricalCrossentropy)
from tensorflow.keras.callbacks import (EarlyStopping, 
                                        ModelCheckpoint)

import keras_tuner as kt

/Users/kavisanthoshkumar/Library/CloudStorage/OneDrive-IllinoisInstituteofTechnology/Tensorflow/tensorflow_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Load Train, Validation and Testing Numpy Arrays

In [3]:
data = np.load("../data/processed/processed_array.npz")

X_train, y_train = data["x_train"], data["y_train"]
X_val, y_val = data["x_val"], data["y_val"]
X_test, y_test = data["x_test"], data["y_test"]

print(f"Shape of X_train : {X_train.shape} and y_train : {y_train.shape}")
print(f"Shape of X_val : {X_val.shape} and y_train : {y_val.shape}")
print(f"Shape of X_test : {X_test.shape} and y_train : {y_test.shape}")

Shape of X_train : (5881, 128, 9) and y_train : (5881, 6)
Shape of X_val : (1471, 128, 9) and y_train : (1471, 6)
Shape of X_test : (2947, 128, 9) and y_train : (2947, 6)


# Tensorflow Dataset

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

2025-07-02 12:21:20.452354: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-07-02 12:21:20.452379: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-07-02 12:21:20.452384: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1751476880.452394  971586 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1751476880.452408  971586 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Keras Tuner Neural Network Architecture

#### Step 1: Neural Network Architecture

In [5]:
class CustomLSTM(Model):

    def __init__(self, hp, num_classes):
        super(CustomLSTM, self).__init__()
        
        # Tunable Parameters
        lstm_units = hp.Int("units", min_value = 32, max_value = 256, step = 32)
    

        # LSTM layer
        self.lstm_layer = LSTM(units = lstm_units, 
                               activation = "tanh",
                               recurrent_activation = 'sigmoid',
                               )
        
        # Dense layers``
        self.dense_layer_1 = Dense(units = 256, activation = "leaky_relu")
        self.dense_layer_2 = Dense(units = 128, activation = "leaky_relu")
        self.dense_layer_3 = Dense(units = 64, activation = "leaky_relu")

        # Output layers
        self.output_layer = Dense(units = num_classes, activation = "softmax")


    def call(self, inputs):

        # lstm layer
        x = self.lstm_layer(inputs)

        # Dense layer
        x = self.dense_layer_1(x)
        x = self.dense_layer_2(x)
        x = self.dense_layer_3(x)

        # Output layer
        out = self.output_layer(x)

        return out 

#### Step 2 : Keras Tuner Hyperband Model

In [6]:
def keras_tuner_hyperband_model(hp):

    # Initiate model
    model = CustomLSTM(hp, num_classes = 6)

    # Compile the model
    hp_optimizers = hp.Choice("optimizers", values = ["Adam", "AdamW"]) 
    model.compile(optimizer = hp_optimizers, loss = CategoricalCrossentropy(), 
                  metrics = ["accuracy", "f1_score"])
    

    return model

#### Step 3: Instantiate the tuner and perform hypertuning

In [10]:
tuner = kt.Hyperband(keras_tuner_hyperband_model, 
                     objective = 'val_accuracy',
                     max_epochs = 3,
                     directory = "../model/",
                     project_name = "har_tuner"
                     )

#### Step 4: Early Stopping

In [11]:
early_stopping = EarlyStopping(
        monitor = "val_loss", 
        verbose = 0,
        mode = "min",
        patience = 5
)

callbacks = [early_stopping]

#### Step 5: Tuner Search

In [14]:
tuner.search(train_dataset,  
             epochs = 50, 
             validation_data = valid_dataset,
             callbacks = callbacks)

# Best Hyperparameters 
best_hps = tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM layer is {best_hps.get("units")} and the best choice of optimizer is
{best_hps.get("optimizers")}
""")


The hyperparameter search is complete. The optimal number of units in the first LSTM layer is 64 and the best choice of optimizer is
AdamW

